# DC_TTS - a test on the pre-trained model which is based on the LJ speech dataset

**!!** **Remember to activate Hardware acceleration** **!!**
Edit → Notebook Settings → select GPU from the hardware accelerator menu

In [ ]:
# clone the repository
!git clone https://github.com/jkuhlemann/dc_tts.git

In [2]:
#create folders necessary for everything to work
!mkdir /content/dc_tts/samples
!mkdir /content/dc_tts/logdir
!mkdir /data/
!mkdir /data/private/
!mkdir /data/private/voice/

In [3]:
# download pre-trained model
import urllib.request

url = "https://www.dropbox.com/s/1oyipstjxh2n5wo/LJ_logdir.tar?dl=1"  

u = urllib.request.urlopen(url)
data = u.read()
u.close()
 
with open('/content/dc_tts/LJ_logdir.tar', "wb") as f :
    f.write(data)

In [4]:
#extract the pre-trained model
import zipfile
with zipfile.ZipFile('/content/dc_tts/LJ_logdir.tar', 'r') as zip_ref:
    zip_ref.extractall('/content/dc_tts/')

In [5]:
#move the pre-trained model data to logdir folder
!mv /content/dc_tts/LJ01-1 /content/dc_tts/logdir
!mv /content/dc_tts/LJ01-2 /content/dc_tts/logdir

In [ ]:
# function to use own sentences for TTS
def sentences_to_synthesize():
  x = input("Do you want to use the example sentences or synthesize your own sentences? Please choose(example/own): ")
  if x == "example":
    print("Good. Just go to the next cell then.")
    pass
  elif x == "own":
    n = 1
    with open('/content/dc_tts/harvard_sentences.txt', 'w') as f:
      f.write('The code ignores the first line in the .txt. This is just a line so that everything works \n')

    own_sentence = input("Alright. Please write your sentence here: ")
    own_sentence = f"{n}. {own_sentence} \n"
    
    with open('/content/dc_tts/harvard_sentences.txt', 'a') as f:
      f.write(own_sentence)
  while True:
    more = input("Would you like to add another sentence? (y/n): ")
    if more == 'y':
      n = n + 1 
      own_sentence = input("Write your sentence here: ")
      own_sentence = f"{n}. {own_sentence} \n"
      with open('/content/dc_tts/harvard_sentences.txt', 'a') as f:
        f.write(own_sentence)
    elif more == 'n':
      return

  return
  
sentences_to_synthesize()

In [ ]:
# manually switch to Tensorflow version 1.x since this is what the repo uses
%tensorflow_version 1.x

In [ ]:
# synthesize the sentences specified in 'harvard_sentences.txt.'
# you will find the resulting .wav files in /content/dc_tts/samples 
# or use the widget implementation below to play the audio
!python /content/dc_tts/synthesize.py

In [ ]:
from IPython.display import Audio, display
# function to play desired audiofile
def play_audio():
  numb = input("Which audiofile would you like to listen to? (Choose number): ")
  filepath = f"/content/dc_tts/samples/{numb}.wav"
  return Audio(filepath)

play_audio()

That's it! We hope you enjoyed the testing the TTS model!